In [ ]:
from ipywidgets import Button, VBox, HBox, HTML
from ipyevents import Event
from random import random
from traitlets import Int, observe

In [ ]:
class MSTile(Button):
    def __init__(self, x, y, parent):
        super().__init__(layout={"width":"30px"})
        self.x = x
        self.y = y
        self.parent = parent
        
        d = Event(  # right-click
            source=self,
            watched_events=['contextmenu'],
            prevent_default_action=True
        )
        def handle_event(event):
            if self.description == "?":
                self.description = " "
                return
            elif self.description.strip():
                return
            elif self.icon == "flag":
                self.icon = ""
                self.description = "?"
                self.parent.flags -= 1
            elif not self.icon:
                self.icon = "flag"
                self.parent.flags += 1

        d.on_dom_event(handle_event)

class MSGrid(VBox):
    flags = Int(0)
    def __init__(self, w, h, difficulty=0.1):
        super().__init__()
        self.w = w
        self.h = h
        self.difficulty = difficulty
        
        def btn_callback(btn):
            if btn.disabled:
                return
            self.n_checked+=1
            btn.disabled = True
            x = btn.x
            y = btn.y
            val = self.counts[y][x]
            if val == "x":
                self.game_over()
            btn.description = val
            if val == "":
                for cell in self.neighbors(y,x):
                    cell.click()
            if self.n_checked + self.n_mines == (w*h):
                self.reset_btn.icon = "smile"
        
        
        self.rows = []
        for y in range(h):
            self.rows.append([])
            for x in range(w):
                btn = MSTile(x, y, self)
                btn.on_click(btn_callback)
                self.rows[-1].append(btn)
        
        
        self.flag_counter = HTML()
        
        self.reset_btn = Button(description="Reset")
        self.reset_btn.on_click(self.reset)
        self.reset_btn.click()
        
        self.children = [HBox([self.reset_btn,self.flag_counter])] + [HBox(row) for row in self.rows]
        
    
    @observe("flags")
    def set_flag_counter(self, _=None):
        self.flag_counter.value = "Remaining: " + str(self.n_mines-self.flags)
    
    def reset(self, btn):
        self.reset_btn.icon = ""
        self.place_mines()
        self.determine_counts()
        self.set_flag_counter()
        
    def place_mines(self):
        self.n_mines = 0
        self.n_checked = 0
        self.flags = 0
        for y in range(self.h):
            for x in range(self.w):
                cell = self.rows[y][x]
                cell.ismine = False
                cell.disabled = False
                cell.description = " "
                cell.icon = ""
                if random() < self.difficulty:
                    self.n_mines += 1
                    self.rows[y][x].ismine = True
    
    def determine_counts(self):
        self.counts = []
        for i,row in enumerate(self.rows):
            self.counts.append([])
            for j,btn in enumerate(row):
                if btn.ismine:
                    self.counts[-1].append("x")
                else:
                    c = 0
                    for cell in self.neighbors(i,j):
                        c += cell.ismine
                    self.counts[-1].append(str(c) if c else "")
        
    def neighbors(self, i, j):
        cells = []
        if i > 0:
            cells.append(self.rows[i-1][j])  # S
            if j > 0:
                cells.append(self.rows[i-1][j-1])  # S-W
            if j < self.w-1:
                cells.append(self.rows[i-1][j+1])  # S-E
        if i<self.h-1:
            cells.append(self.rows[i+1][j])  # N
            if j > 0:
                cells.append(self.rows[i+1][j-1])  # N-W
                
            if j < self.w-1:
                cells.append(self.rows[i+1][j+1])  # N-E
        if j > 0:
            cells.append(self.rows[i][j-1])  # W
        if j < self.w-1:
            cells.append(self.rows[i][j+1])  # E
        return [x for x in cells if not x.disabled]
    
    def game_over(self):
        self.reset_btn.icon="frown"
        for r in self.rows:
            for btn in r:
                if btn.ismine:
                    btn.click()

In [ ]:
MSGrid(20,20,difficulty = .18)